# NLP Class Final Project 
##  Yash Verma 

In [1]:
import pandas as pd
import re 
import nltk as nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import time
import math
import string
import gensim
import pyLDAvis
import seaborn as sns 
import matplotlib.pyplot as plt
#import pyLDAvis.gensim
#import pyLDAvis.gensim_models as gensimvis

from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore

In [2]:
import pandarallel
import multiprocessing
from pandarallel import pandarallel

num_processors = multiprocessing.cpu_count()

pandarallel.initialize(nb_workers=num_processors, use_memory_fs=False)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [3]:
# !pip install pyarrow

  Obtaining dependency information for pyarrow from https://files.pythonhosted.org/packages/dc/45/31441c988329afed625a791a7d78f1cf2fcb40dcc86a1d61e081287516a8/pyarrow-12.0.1-cp39-cp39-win_amd64.whl.metadata
   ---------------------------------------- 21.5/21.5 MB 25.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%%time

df_news_final_project = pd.read_parquet(r'C:\Users\verma\OneDrive - The University of Chicago\Desktop\MSCA\QUARTER 4 SUMMER\NLP\FINAL PROJECT\news_final_project.parquet', engine='pyarrow')
df_news_final_project.shape

CPU times: total: 17.2 s
Wall time: 36 s


(199208, 5)

In [4]:
df_news_final_project.head(5)

,url,date,language,title,text
0,http://newsparliament.com/2020/02/27/children-...,2020-02-27,en,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...
1,http://thedailycougar.com/2021/11/24/college-o...,2021-11-24,en,College of Technology to launch AI Retail Lab ...,\nCollege of Technology to launch AI Retail L...
2,http://www.desototimes.com/news/child-actor-cl...,2022-06-09,en,Child actor Claude Jarman Jr. looks back on hi...,\nChild actor Claude Jarman Jr. looks back on ...
3,http://www.mysmartrend.com/news-briefs/technic...,2020-04-17,en,Cr Bard Inc Has Returned 48.9% Since SmarTrend...,\n\nCr Bard Inc Has Returned 48.9% Since SmarT...
4,http://www.peripherals.consumerelectronicsnet....,2021-02-25,en,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...


### Data Cleaning 

In [5]:
# The articles are all in English 
df_news_final_project["language"].unique()

array(['en'], dtype=object)

In [6]:
# Dropping redudant columns 
columns_to_drop = ['url', 'language']
df = df_news_final_project.drop(columns = columns_to_drop)

In [7]:
df.head(5)

,date,title,text
0,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...
1,2021-11-24,College of Technology to launch AI Retail Lab ...,\nCollege of Technology to launch AI Retail L...
2,2022-06-09,Child actor Claude Jarman Jr. looks back on hi...,\nChild actor Claude Jarman Jr. looks back on ...
3,2020-04-17,Cr Bard Inc Has Returned 48.9% Since SmarTrend...,\n\nCr Bard Inc Has Returned 48.9% Since SmarT...
4,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...


In [8]:
df.dtypes

date     object
title    object
text     object
dtype: object

The data type of the date column is object so we should change it to the datetime format

In [9]:
# Convert the 'date_column' to datetime data type
df['date'] = pd.to_datetime(df['date'])

In [10]:
df.head(5)

,date,title,text
0,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...
1,2021-11-24,College of Technology to launch AI Retail Lab ...,\nCollege of Technology to launch AI Retail L...
2,2022-06-09,Child actor Claude Jarman Jr. looks back on hi...,\nChild actor Claude Jarman Jr. looks back on ...
3,2020-04-17,Cr Bard Inc Has Returned 48.9% Since SmarTrend...,\n\nCr Bard Inc Has Returned 48.9% Since SmarT...
4,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...


In [11]:
df.dtypes

date     datetime64[ns]
title            object
text             object
dtype: object

In [12]:
# Splitting the date into days months and years fir future analysis use

df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day
df["dayofweek"] = df["date"].dt.dayofweek

In [13]:
df.head(5)

,date,title,text,year,month,day,dayofweek
0,2020-02-27,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,2020,2,27,3
1,2021-11-24,College of Technology to launch AI Retail Lab ...,\nCollege of Technology to launch AI Retail L...,2021,11,24,2
2,2022-06-09,Child actor Claude Jarman Jr. looks back on hi...,\nChild actor Claude Jarman Jr. looks back on ...,2022,6,9,3
3,2020-04-17,Cr Bard Inc Has Returned 48.9% Since SmarTrend...,\n\nCr Bard Inc Has Returned 48.9% Since SmarT...,2020,4,17,4
4,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...,2021,2,25,3


In [14]:
def clean_text(text):
    import re
    # Removing unimportant characters (newlines,tabs)
    clean = re.sub(r'\n+|\t+|\s+$|^\s+', '', text) 
    
    # Removing unimportant characters
    clean = re.sub(r'[^a-zA-Z0-9 @ . , : - _ ]', '', clean) 
    
    # Remove URLs and email addresses
    clean = re.sub(r'http\S+|www.\S+|\S+@\S+', '', clean) 
    
    # Remove non-printable characters
    clean = ''.join(filter(lambda x: x.isprintable(), clean)) 
    
    # Remove non-ASCII characters
    clean = ''.join(filter(lambda x: ord(x) < 128, clean)) 
    
    # Replace multiple spaces with a single space
    clean = re.sub(r'\s+', ' ', clean)
    
    return clean

In [15]:
# Making new columns for the new cleaned text and title

%time df['clean_title'] = df['title'].parallel_apply(clean_text)

CPU times: total: 219 ms
Wall time: 6.32 s


In [16]:
%time df['clean_text'] = df['text'].parallel_apply(clean_text)

CPU times: total: 22.4 s
Wall time: 4min 1s


In [17]:
pd.set_option('display.max_colwidth', 1000)
df['clean_text'].head(1)

0    Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot News Parliament Skip to contentThursday, February 27, 2020Latest:Mansplaining in conferences: How can we get him to forestallDrax power station to cease burning coal in March 2021Coronavirus Could Explode in the U.S. Overnight Like it Did in ItalyCoronavirus: Dettol sales surge as markets fall againLevi Strauss marks the next phase in corporate paid leave policiesNews ParliamentGet the Real Story NewsTechnologyServicesHealthcareWorldBusinessEntertainment World Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot February 27, 2020February 27, 2020admin 0 Comments Author Recent Posts admin Latest posts by admin see all Mansplaining in conferences: How can we get him to forestall February 27, 2020 Coronavirus Could Explode in the U.S. Overnight Like it Did in Italy February 27, 2020 Levi Strauss marks the next phase in corporate paid leave p

In [18]:
df.shape

(199208, 9)

### Using the keywords related to clear out the irrelevant articles that are not related to Aritificial Intelligence 

In [19]:
keywords = [
    'data science',
    'machine learning',
    'artificial intelligence',
    'natural language processing',
    'computer vision',
    'big data',
    'data mining',
    'deep learning',
    'reinforcement learning',
    'computer vision',
    'natural language generation',
    'text analytics',
    'image recognition',
    'speech recognition',
    'recommendation systems',
    'fraud detection',
    'predictive analytics',
    'neural networks',
    'pattern recognition',
    'data engineering',
    'feature extraction',
    'anomaly detection',
    'time series analysis',
    'cognitive computing',
    'chatbots',
    'predictive modeling',
    'sentiment analysis',
    'bayesian statistics',
    'decision trees',
    'support vector machines',
    'ensemble methods',
    'model deployment',
    'neural language translation',
    'dimensionality reduction',
    'cloud computing',
    'internet of things',
    'blockchain applications',
    'cybersecurity and AI',
    'explainable AI',
    'human-AI interaction',
    'AI ethics',
    'quantum computing (AI applications)'
]

In [20]:
%%time

filtered_df = df[df['clean_text'].str.contains('|'.join(keywords), flags=re.IGNORECASE)]

<timed exec>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.


CPU times: total: 5min 7s
Wall time: 7min 15s


In [21]:
filtered_df.shape

(157235, 9)

In [24]:
filtered_df.head(2)

,date,title,text,year,month,day,dayofweek,clean_title,clean_text
1,2021-11-24,College of Technology to launch AI Retail Lab - The Cougar,"\nCollege of Technology to launch AI Retail Lab - The Cougar\nside bar\n\n\nHome\nCoronavirus\nNews\n\nAcademics & Research\nActivities & Organizations\nAdministration\nAlumni\nState\nStudent Government\nFacilities\nUH System\nCampus\nCity\nConstruction\nCrime\nFaculty & Staff\nNation\n\n\nSports\n\nBaseball\nCommentary\nCross Country\nFootball\nGolf\nMen’s Basketball\nSoccer\nSoftball\nSwimming & Diving\nTennis\nTrack & Field\nVolleyball\nWomen’s Basketball\n\n\nOpinion\n\nStaff Editorial\nWeb Exclusive\n\n\nLife + Arts\n\nSocial Media\n\n\nPrint Edition\nClassifieds\nAbout\n\nContact us\nAdvertising\nFeedback\nGet in the paper\nFind The Cougar\n\n\nApply\n \n\n \n \n\n\n \n\n#REaDAllOver\n\nIn Print Every Wednesday\nIn Your Newsfeed fb.com/thedailycougar\nOn Twitter @thedailycougar\nOn Instagram@thedailycougar\nIn Your Inbox Email subscription\nJoin the staff Apply for a position\n\n\nWednesday, November 24, 2021\n\nHome \nCoronavirus \nNews 6d707f\n\nAcademics & Research \nActi...",2021,11,24,2,College of Technology to launch AI Retail Lab The Cougar,"College of Technology to launch AI Retail Lab The Cougarside barHomeCoronavirusNewsAcademics ResearchActivities OrganizationsAdministrationAlumniStateStudent GovernmentFacilitiesUH SystemCampusCityConstructionCrimeFaculty StaffNationSportsBaseballCommentaryCross CountryFootballGolfMens BasketballSoccerSoftballSwimming DivingTennisTrack FieldVolleyballWomens BasketballOpinionStaff EditorialWeb ExclusiveLife ArtsSocial MediaPrint EditionClassifiedsAboutContact usAdvertisingFeedbackGet in the paperFind The CougarApply REaDAllOverIn Print Every WednesdayIn Your Newsfeed fb.comthedailycougarOn Twitter @thedailycougarOn Your Inbox Email subscriptionJoin the staff Apply for a positionWednesday, November 24, 2021Home Coronavirus News 6d707fAcademics Research Activities Organizations Administration Alumni State Student Government Facilities UH System Campus City Construction Crime Faculty Staff Nation Sports BC1128Baseball Commentary Cross Country Football Golf Mens Basketball Soccer Softba..."
4,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision – Consumer Electronics Net,"\n\nMulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision – Consumer Electronics Net\n\n \nSkip to content\n\nConsumer Electronics Net\n\nPrimary Menu\n\nConsumer Electronics Net\n\nSearch for:\n \nHomeNewsMulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision \n \n News\n \n \nMulticoreWare Inc. Becomes CEVA’s Trusted Partner for Imaging & Computer Vision\n 13 mins ago \n \n\nMulticoreWare Inc., joins the CEVAnet™ Partner Program – a comprehensive partner ecosystem to form a robust infrastructure around CEVA’s IP offerings. \n\n\nSAN JOSE, Calif.–(BUSINESS WIRE)–#ADAS–MulticoreWare Inc., joins CEVA, the leading licensor of sensor-hub Digital Signal Processors, AI engines and complementary framework & various software SDK’s, as their trusted partner to enable services on all computing platforms by implementing optimized kernels and frameworks ...",2021,2,25,3,MulticoreWare Inc. Becomes CEVAs Trusted Partner for Imaging Computer Vision Consumer Electronics Net,"MulticoreWare Inc. Becomes CEVAs Trusted Partner for Imaging Computer Vision Consumer Electronics Net Skip to contentConsumer Electronics NetPrimary MenuConsumer Electronics NetSearch for: HomeNewsMulticoreWare Inc. Becomes CEVAs Trusted Partner for Imaging Computer Vision News MulticoreWare Inc. Becomes CEVAs Trusted Partner for Imaging Computer Vision 13 mins ago MulticoreWare Inc., joins the CEVAnet Partner Program a comprehensive partner ecosystem to form a robust infrastructure around CEVAs IP offerings. SAN JOSE, Calif.BUSINESS WIREADASMulticoreWare Inc., joins CEVA, the leading licensor of sensorhub Digital Signal Processors, AI

In [25]:
output_path = r"C:\Users\verma\OneDrive - The University of Chicago\Desktop\MSCA\QUARTER 4 SUMMER\NLP\FINAL PROJECT/data.parquet"
df.to_parquet(output_path, index=False)